In [12]:
import requests
import json
import os
import numpy as np
import pandas as pd

API_KEY = os.getenv("API_KEY")
url = 'https://data-service.beatthebookie.blog/data'
headers = {"x-api-key": API_KEY}
params = {'division':'Premier League'}
response = requests.get(url, headers=headers, params=params)
json_str_prem = response.content.decode('utf-8')
prem_df= pd.read_json(json_str_prem)

      match_date    home_team  away_team  home_goals  home_xgoals  away_goals  \
3815  2024-08-24       Fulham  Leicester           2      1.96218           1   
3816  2024-08-24     Man City    Ipswich           4      3.07818           1   
3817  2024-08-25  Bournemouth  Newcastle           1      2.53513           1   
3818  2024-08-25       Wolves    Chelsea           2      2.41810           6   
3819  2024-08-25    Liverpool  Brentford           2      2.71877           0   

      away_xgoals  
3815     0.857005  
3816     0.479999  
3817     1.892870  
3818     2.029350  
3819     0.457239  


C:\Users\Owner\AppData\Local\Temp\ipykernel_7656\3235614769.py:13: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  prem_df= pd.read_json(json_str_prem)


In [13]:
params = {'division':'Championship'}
response = requests.get(url, headers=headers, params=params)
json_str_champ = response.content.decode('utf-8')
champ_df= pd.read_json(json_str_champ)

      match_date home_team         away_team  home_goals  home_xgoals  \
3887  2024-08-24   Preston             Luton           1          0.3   
3888  2024-08-24   Watford             Derby           2          1.1   
3889  2024-08-24   Norwich  Sheffield United           1          1.0   
3890  2024-08-24       QPR   Plymouth Argyle           1          2.2   
3891  2024-08-25   Swansea           Cardiff           1          1.3   

      away_goals  away_xgoals  
3887           0          1.6  
3888           1          0.7  
3889           1          0.8  
3890           1          0.1  
3891           1          0.7  


C:\Users\Owner\AppData\Local\Temp\ipykernel_7656\3483202742.py:4: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  champ_df= pd.read_json(json_str_champ)


In [41]:
from datetime import datetime

df = pd.concat([prem_df, champ_df])

# Separates out the home vs away and allows every team to have an individual row of data. Allows us to feature engineer home effect easier.
home_df = df[["division", "season", "match_date",
             "home_team", "home_avg_market_value", "home_goals", "home_xgoals",
             "away_team", "away_avg_market_value", "away_goals", "away_xgoals"]].copy()
home_df["home?"] = 1

home_df = home_df.rename(columns={"home_team":"team", "home_avg_market_value":"avg_market_value", "home_goals":"goals", "home_xgoals":"xG",
             "away_team":"opponent_team", "away_avg_market_value":"opponent_avg_market_value", "away_goals":"opponent_goals", "away_xgoals": "opponent_xG"})

away_df = df[["division", "season", "match_date",
                "away_team", "away_avg_market_value", "away_goals", "away_xgoals",
                "home_team", "home_avg_market_value", "home_goals", "home_xgoals"]].copy()
away_df["home?"] = 0

away_df = away_df.rename(columns={"away_team":"team", "away_avg_market_value":"avg_market_value", "away_goals":"goals", "away_xgoals":"xG",
             "home_team":"opponent_team", "home_avg_market_value":"opponent_avg_market_value", "home_goals":"opponent_goals", "home_xgoals": "opponent_xG"})

df = pd.concat([away_df, home_df])
df.sort_values(["match_date", "division"], inplace=True)

# Create a unique identifier for each game, so even though matches are now duplicated, they have the same id.
df["match_id"] = df.apply(
    lambda row: "-".join(sorted([row["division"], str(row["season"]), str(row['match_date']), row['team'], row['opponent_team']])),
    axis=1
) # This applies a function to each row that creates a list out of the above columns, sorts and sorts them in alphabetical order.
  # It then joins the contents of the strings to create a unique ID like: 2014-2014-08-08-Ligue 1-Paris SG-Reims

df["match_id"] = df["match_id"].apply(hash) # This then hashes the string ID into a numeric value.

df["match_date"] = pd.to_datetime(df["match_date"])
df["days_since"] = (datetime.now() - df["match_date"]).dt.days.astype(float)

In [42]:
#TODO:  Placeholder to transform target variable to log scaled.